In [1]:
# upgrade pip, make sure you are in the correct directory, and make sure you have the right python
#!python -m pip install --upgrade pip
import os
print(os.getcwd())
os.listdir()
#os.chdir('C:\Users\Christopher\Desktop\DataWarehouseProject\Design\Dimension_Color')
#os.chdir('C:\Users\Christopher\Desktop\Data Warehouse Project\Design\Dimension_Color')
print(os.getcwd())
# Check your version of python to make sure you are running python 3
import platform
print('Pyhton Version: ' + platform.python_version())


d:\django_dashboard_corona\secondDashboard\jcwf_report_credit_analysis
d:\django_dashboard_corona\secondDashboard\jcwf_report_credit_analysis
Pyhton Version: 3.8.10


In [5]:
import psycopg2
import pandas as pd
from pandas import DataFrame
import datetime as dt

# writing query

def simple_query(customer_key, start_date, end_date):
    start_date = str('\'')+start_date+str('\'')
    end_date = str('\'')+end_date+str('\'')
    if len(customer_key)>1:
        customer_filter = (str("""
        DROP TABLE IF EXISTS  jcwf_temp_table_t;
        SELECT * INTO TABLE jcwf_temp_table_t FROM jcwf_temp_table WHERE sales_order_item_customer_id_foreign_key = """)
        +
        str(customer_key)
        +
        str(""";"""))
    else:
        customer_filter =str("""
        DROP TABLE IF EXISTS  jcwf_temp_table_t;
        SELECT * INTO TABLE jcwf_temp_table_t FROM jcwf_temp_table;""")

    query = (
    str("""

    --This is a query that creates a tempeorary table of the Detail line items data, enriched with  product data and color data
    --Author: Christopher Gonzalez
    --SELECT * FROM jcwf_tran_sales_order_item

    DROP TABLE IF EXISTS  jcwf_temp_table;

    SELECT

    sales_order_item_status 
    , sales_order_item_product_foreign_key 
    , product_description
    , sales_order_item_terms 
    , sales_order_item_order_item  
    , sales_order_item_po 
    , sales_order_item_customer_id_foreign_key 
    , sales_order_item_sidemark 
    , sales_order_item_entered 
    , sales_order_item_credit_ok 
    , sales_order_item_printed 
    , sales_order_item_labels 
    , sales_order_item_packed
    , sales_order_item_shipped_date 
    , sales_order_item_required 
    , sales_order_item_canceled
    , sales_order_item_model
    , sales_order_item_color_foreign_key 
    , sales_order_item_width
    , sales_order_item_height 
    , sales_order_item_ordered
    , sales_order_item_shipped_quantity
    , sales_order_item_net_sale  

    INTO TABLE jcwf_temp_table
    FROM jcwf_fact_sales_order_item
    JOIN jcwf_dim_product
    ON jcwf_fact_sales_order_item.sales_order_item_product_foreign_key = jcwf_dim_product.product_key
    """)
    +
    str(
    """
    WHERE sales_order_item_entered >= 
    """)
    +
    str(start_date)
    +
    str(
    """
    AND sales_order_item_entered <=
    """)
    +
    str(end_date)
    # +
    # """
    # AND sales_order_item_customer_id_foreign_key == ""
    # """
    +
    str(""";""")
    +
    # str("""SELECT * FROM jcwf_temp_table;""")
    # +
    str(customer_filter)

    # +
    # str("""
    # SELECT  * FROM jcwf_temp_table_t;
    # """)
    +
    str("""
    DROP TABLE IF EXISTS  jcwf_temp_table_2;

    SELECT  

    sales_order_item_status 
    , sales_order_item_product_foreign_key 
    , product_description
    , sales_order_item_terms 
    , sales_order_item_order_item  
    , sales_order_item_po 
    , sales_order_item_customer_id_foreign_key 
    , Customer_Account
    , Customer_Name
    , sales_order_item_sidemark 
    , sales_order_item_entered 
    , sales_order_item_credit_ok 
    , sales_order_item_printed 
    , sales_order_item_labels 
    , sales_order_item_packed
    , sales_order_item_shipped_date 
    , sales_order_item_required 
    , sales_order_item_canceled
    , sales_order_item_model
    , sales_order_item_color_foreign_key 
    , sales_order_item_width
    , sales_order_item_height 
    , sales_order_item_ordered
    , sales_order_item_shipped_quantity
    , sales_order_item_net_sale  

    INTO TABLE jcwf_temp_table_2
    FROM jcwf_temp_table_t
    JOIN jcwf_dim_customer
    ON jcwf_temp_table_t.sales_order_item_customer_id_foreign_key = jcwf_dim_customer.Customer_key;""")
    # +
    # str("""SELECT * FROM jcwf_temp_table_2;""")
    +
    str("""
    DROP TABLE IF EXISTS  jcwf_temp_table_3;

    SELECT  

    sales_order_item_status 
    , sales_order_item_product_foreign_key 
    , product_description
    , sales_order_item_terms 
    , sales_order_item_order_item  
    , sales_order_item_po 
    , sales_order_item_customer_id_foreign_key 
    , Customer_Account
    , Customer_Name
    , sales_order_item_sidemark 
    , sales_order_item_entered 
    , sales_order_item_credit_ok 
    , sales_order_item_printed 
    , sales_order_item_labels 
    , sales_order_item_packed
    , sales_order_item_shipped_date 
    , sales_order_item_required 
    , sales_order_item_canceled
    , sales_order_item_model
    , sales_order_item_color_foreign_key 
    , color_name
    , color_description
    , sales_order_item_width
    , sales_order_item_height 
    , sales_order_item_ordered
    , sales_order_item_shipped_quantity
    , sales_order_item_net_sale  

    INTO TABLE jcwf_temp_table_3
    FROM jcwf_temp_table_2
    JOIN jcwf_dim_color
    ON jcwf_temp_table_2.sales_order_item_color_foreign_key = jcwf_dim_color.color_key;""")
    +
    str("""SELECT * FROM jcwf_temp_table_3;""")
    )
    # print(query)

    # importing data from the data warehouse using psycopg2
    conn = psycopg2.connect("host='econometricdatasolutionsdb.postgres.database.azure.com' dbname=postgres user='Christopher@econometricdatasolutionsdb' password='Darkknight17!'")
    cur = conn.cursor()
    cur.execute(query) 
    rows = cur.fetchall()

    # print ("\nShow me the databases:\n")
    # for row in rows:  
    #     print ("   ", row)

    # rows = cur.fetchall()
    
    df = DataFrame(rows,columns=[
    
      'sales_order_item_status' 
    , 'sales_order_item_product_foreign_key' 
    , 'product_description'
    , 'sales_order_item_terms' 
    , 'sales_order_item_order_item'  
    , 'sales_order_item_po' 
    , 'sales_order_item_customer_id_foreign_key' 
    , 'Customer_Account'
    , 'Customer_Name'
    , 'sales_order_item_sidemark' 
    , 'sales_order_item_entered' 
    , 'sales_order_item_credit_ok' 
    , 'sales_order_item_printed' 
    , 'sales_order_item_labels' 
    , 'sales_order_item_packed'
    , 'sales_order_item_shipped_date' 
    , 'sales_order_item_required' 
    , 'sales_order_item_canceled'
    , 'sales_order_item_model'
    , 'sales_order_item_color_foreign_key' 
    , 'color_name'
    , 'color_description'
    , 'sales_order_item_width'
    , 'sales_order_item_height' 
    , 'sales_order_item_ordered'
    , 'sales_order_item_shipped_quantity'
    , 'sales_order_item_net_sale'  
    ])
    
    conn.commit()
    conn.close()
    return(df)

simple_query('537','2021-08-18', '2021-10-18')


,sales_order_item_status,sales_order_item_product_foreign_key,product_description,sales_order_item_terms,sales_order_item_order_item,sales_order_item_po,sales_order_item_customer_id_foreign_key,Customer_Account,Customer_Name,sales_order_item_sidemark,...,sales_order_item_canceled,sales_order_item_model,sales_order_item_color_foreign_key,color_name,color_description,sales_order_item_width,sales_order_item_height,sales_order_item_ordered,sales_order_item_shipped_quantity,sales_order_item_net_sale
0,20 CREDIT APPROVED,56,HONEYCOMB,COD,298706-1,None,537,CLA03,CLASSIC ULTRASONIC BLINDS,BRYAN & KRISTEN,...,None,CORDLESS,1522,T05-503,SNOW - 3/4 SC,22.25,46.5,1,0,101.37
1,5 PRODUCTION HOLD,45,ROLLER SHADES,COD,299011-5,None,537,CLA03,CLASSIC ULTRASONIC BLINDS,HECTOR - UNIT 1,...,None,CLUTCH,1645,DISC,DISCONTINUED,22.375,46.25,1,0,123.36
2,5 PRODUCTION HOLD,45,ROLLER SHADES,COD,299011-6,None,537,CLA03,CLASSIC ULTRASONIC BLINDS,HECTOR - UNIT 1,...,None,CLUTCH,1645,DISC,DISCONTINUED,22.375,46.25,1,0,123.36
3,5 PRODUCTION HOLD,45,ROLLER SHADES,COD,299011-7,None,537,CLA03,CLASSIC ULTRASONIC BLINDS,HECTOR - UNIT 1,...,None,CLUTCH,1645,DISC,DISCONTINUED,22.75,58.5,1,0,128.29
4,5 PRODUCTION HOLD,45,ROLLER SHADES,COD,299011-8,None,537,CLA03,CLASSIC ULTRASONIC BLINDS,HECTOR - UNIT 1,...,None,CLUTCH,1645,DISC,DISCONTINUED,48,58.5,1,0,197.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,20 CREDIT APPROVED,45,ROLLER SHADES,COD,298627-3,None,537,CLA03,CLASSIC ULTRASONIC BLINDS,JORGE & LOURDES,...,None,CORDLESS,3953,ALE111,Alexa White,91.5,44,1,0,238.96
113,30 BATCHED,45,ROLLER SHADES,COD,298179-1,None,537,CLA03,CLASSIC ULTRASONIC BLINDS,LINDA JAYCE,...,None,CLUTCH,3954,ALE112,Alexa Praline,107,52,1,0,217.03
114,30 BATCHED,45,ROLLER SHADES,COD,298179-2,None,537,CLA03,CLASSIC ULTRASONIC BLINDS,LINDA JAYCE,...,None,CLUTCH,3954,ALE112,Alexa Praline,43.25,52.25,1,0,111.29
115,30 BATCHED,45,ROLLER SHADES,COD,298179-3,None,537,CLA03,CLASSIC ULTRASONIC BLINDS,LINDA JAYCE,...,None,CLUTCH,3954,ALE112,Alexa Praline,43.5,52.5,1,0,111.29


In [11]:


df = simple_query('537','2021-08-18', '2021-10-18')
print(df)

    sales_order_item_status  sales_order_item_product_foreign_key  \
0        20 CREDIT APPROVED                                    56   
1         5 PRODUCTION HOLD                                    45   
2         5 PRODUCTION HOLD                                    45   
3         5 PRODUCTION HOLD                                    45   
4         5 PRODUCTION HOLD                                    45   
..                      ...                                   ...   
112      20 CREDIT APPROVED                                    45   
113              30 BATCHED                                    45   
114              30 BATCHED                                    45   
115              30 BATCHED                                    45   
116              30 BATCHED                                    45   

    product_description sales_order_item_terms sales_order_item_order_item  \
0             HONEYCOMB                    COD                    298706-1   
1         ROLLE

In [18]:
import psycopg2
import pandas as pd
from pandas import DataFrame
import datetime as dt

# writing query

def report_query(customer_key, start_date, end_date):
    start_date = str('\'')+str(start_date)+str('\'')
    end_date = str('\'')+str(end_date)+str('\'')
    if len(str(customer_key))>1:
        customer_filter = (str("""
        DROP TABLE IF EXISTS  jcwf_temp_table_t;
        SELECT * INTO TABLE jcwf_temp_table_t FROM jcwf_temp_table WHERE sales_order_item_customer_id_foreign_key = """)
        +
        str(customer_key)
        +
        str(""";"""))
    else:
        customer_filter =str("""
        DROP TABLE IF EXISTS  jcwf_temp_table_t;
        SELECT * INTO TABLE jcwf_temp_table_t FROM jcwf_temp_table;""")

    query = (
    str("""

    --This is a query that creates a tempeorary table of the Detail line items data, enriched with  product data and color data
    --Author: Christopher Gonzalez
    --SELECT * FROM jcwf_tran_sales_order_item

    DROP TABLE IF EXISTS  jcwf_temp_table;

    SELECT

    sales_order_item_status 
    , sales_order_item_product_foreign_key 
    , product_description
    , sales_order_item_terms 
    , sales_order_item_order_item  
    , sales_order_item_po 
    , sales_order_item_customer_id_foreign_key 
    , sales_order_item_sidemark 
    , sales_order_item_entered 
    , sales_order_item_credit_ok 
    , sales_order_item_printed 
    , sales_order_item_labels 
    , sales_order_item_packed
    , sales_order_item_shipped_date 
    , sales_order_item_required 
    , sales_order_item_canceled
    , sales_order_item_model
    , sales_order_item_color_foreign_key 
    , sales_order_item_width
    , sales_order_item_height 
    , sales_order_item_ordered
    , sales_order_item_shipped_quantity
    , sales_order_item_net_sale  

    INTO TABLE jcwf_temp_table
    FROM jcwf_fact_sales_order_item
    JOIN jcwf_dim_product
    ON jcwf_fact_sales_order_item.sales_order_item_product_foreign_key = jcwf_dim_product.product_key
    """)
    +
    str(
    """
    WHERE sales_order_item_entered >= 
    """)
    +
    str(start_date)
    +
    str(
    """
    AND sales_order_item_entered <=
    """)
    +
    str(end_date)
    # +
    # """
    # AND sales_order_item_customer_id_foreign_key == ""
    # """
    +
    str(""";""")
    +
    # str("""SELECT * FROM jcwf_temp_table;""")
    # +
    str(customer_filter)

    # +
    # str("""
    # SELECT  * FROM jcwf_temp_table_t;
    # """)
    +
    str("""
    DROP TABLE IF EXISTS  jcwf_temp_table_2;

    SELECT  

    sales_order_item_status 
    , sales_order_item_product_foreign_key 
    , product_description
    , sales_order_item_terms 
    , sales_order_item_order_item  
    , sales_order_item_po 
    , sales_order_item_customer_id_foreign_key 
    , Customer_Account
    , Customer_Name
    , sales_order_item_sidemark 
    , sales_order_item_entered 
    , sales_order_item_credit_ok 
    , sales_order_item_printed 
    , sales_order_item_labels 
    , sales_order_item_packed
    , sales_order_item_shipped_date 
    , sales_order_item_required 
    , sales_order_item_canceled
    , sales_order_item_model
    , sales_order_item_color_foreign_key 
    , sales_order_item_width
    , sales_order_item_height 
    , sales_order_item_ordered
    , sales_order_item_shipped_quantity
    , sales_order_item_net_sale  

    INTO TABLE jcwf_temp_table_2
    FROM jcwf_temp_table_t
    JOIN jcwf_dim_customer
    ON jcwf_temp_table_t.sales_order_item_customer_id_foreign_key = jcwf_dim_customer.Customer_key;""")
    # +
    # str("""SELECT * FROM jcwf_temp_table_2;""")
    +
    str("""
    DROP TABLE IF EXISTS  jcwf_temp_table_3;

    SELECT  

    sales_order_item_status 
    , sales_order_item_product_foreign_key 
    , product_description
    , sales_order_item_terms 
    , sales_order_item_order_item  
    , sales_order_item_po 
    , sales_order_item_customer_id_foreign_key 
    , Customer_Account
    , Customer_Name
    , sales_order_item_sidemark 
    , sales_order_item_entered 
    , sales_order_item_credit_ok 
    , sales_order_item_printed 
    , sales_order_item_labels 
    , sales_order_item_packed
    , sales_order_item_shipped_date 
    , sales_order_item_required 
    , sales_order_item_canceled
    , sales_order_item_model
    , sales_order_item_color_foreign_key 
    , color_name
    , color_description
    , sales_order_item_width
    , sales_order_item_height 
    , sales_order_item_ordered
    , sales_order_item_shipped_quantity
    , sales_order_item_net_sale  

    INTO TABLE jcwf_temp_table_3
    FROM jcwf_temp_table_2
    JOIN jcwf_dim_color
    ON jcwf_temp_table_2.sales_order_item_color_foreign_key = jcwf_dim_color.color_key;""")
    +
    str("""SELECT * FROM jcwf_temp_table_3;""")
    )
    # print(query)

    # importing data from the data warehouse using psycopg2
    conn = psycopg2.connect("host='econometricdatasolutionsdb.postgres.database.azure.com' dbname=postgres user='Christopher@econometricdatasolutionsdb' password='Darkknight17!'")
    cur = conn.cursor()
    cur.execute(query) 
    rows = cur.fetchall()

    df = DataFrame(rows,columns=[
    
      'sales_order_item_status' 
    , 'sales_order_item_product_foreign_key' 
    , 'product_description'
    , 'sales_order_item_terms' 
    , 'sales_order_item_order_item'  
    , 'sales_order_item_po' 
    , 'sales_order_item_customer_id_foreign_key' 
    , 'Customer_Account'
    , 'Customer_Name'
    , 'sales_order_item_sidemark' 
    , 'sales_order_item_entered' 
    , 'sales_order_item_credit_ok' 
    , 'sales_order_item_printed' 
    , 'sales_order_item_labels' 
    , 'sales_order_item_packed'
    , 'sales_order_item_shipped_date' 
    , 'sales_order_item_required' 
    , 'sales_order_item_canceled'
    , 'sales_order_item_model'
    , 'sales_order_item_color_foreign_key' 
    , 'color_name'
    , 'color_description'
    , 'sales_order_item_width'
    , 'sales_order_item_height' 
    , 'sales_order_item_ordered'
    , 'sales_order_item_shipped_quantity'
    , 'sales_order_item_net_sale'  
    ])
    
    conn.commit()
    conn.close()
    return(df)

# if __name__ == '__main__':
#     try:
#         arg = sys.argv[1]
#     except IndexError:
#         arg = None

#     return_val = report_query(arg)


import psycopg2
import pandas as pd
from pandas import DataFrame
import datetime as dt

# writing query

def customer_foreign_key_query(Customer_Name):
    # if len(Customer_Name)>1:
    Customer_Name = str('\'')+str(Customer_Name)+str('\'')
    
    customer_filter = (str("""
    DROP TABLE IF EXISTS  jcwf_temp_table_customer_foreign_key_look_up;
    SELECT customer_key INTO TABLE jcwf_temp_table_customer_foreign_key_look_up FROM jcwf_dim_customer WHERE customer_name = """)
    +
    str(Customer_Name)
    +
    str(""";""")
        +
    str("""SELECT * FROM jcwf_temp_table_customer_foreign_key_look_up;""")
    )
    # print(customer_filter)

    conn = psycopg2.connect("host='econometricdatasolutionsdb.postgres.database.azure.com' dbname=postgres user='Christopher@econometricdatasolutionsdb' password='Darkknight17!'")
    cur = conn.cursor()
    cur.execute(customer_filter) 
    rows = cur.fetchall()

    df = DataFrame(rows,columns=[
        'sales_order_item_status'  
     ])
    # print(df.iloc[0]['sales_order_item_status']) 

    conn.commit()
    conn.close()
    return(df.iloc[0]['sales_order_item_status'])

In [20]:
Customer_Name = 'VK CUSTOM LIVING'
customer_key = customer_foreign_key_query(Customer_Name)
report_query(customer_key,'2020-08-18', '2021-10-18')

,sales_order_item_status,sales_order_item_product_foreign_key,product_description,sales_order_item_terms,sales_order_item_order_item,sales_order_item_po,sales_order_item_customer_id_foreign_key,Customer_Account,Customer_Name,sales_order_item_sidemark,...,sales_order_item_canceled,sales_order_item_model,sales_order_item_color_foreign_key,color_name,color_description,sales_order_item_width,sales_order_item_height,sales_order_item_ordered,sales_order_item_shipped_quantity,sales_order_item_net_sale
0,99 VOIDED,56,HONEYCOMB,None,294616-1,None,2117,1391,VK CUSTOM LIVING,BARB-SAMUELS(SNOW),...,None,EZLIFT,1522,T05-503,SNOW - 3/4 SC,30.5,71,5,0,676
1,99 VOIDED,56,HONEYCOMB,None,294616-2,None,2117,1391,VK CUSTOM LIVING,BARB-SAMUELS(SNOW),...,None,EZLIFT,1522,T05-503,SNOW - 3/4 SC,22.5,47,1,0,79.82
2,99 VOIDED,56,HONEYCOMB,None,294617-2,None,2117,1391,VK CUSTOM LIVING,SAMUELS-SUGAR CANE,...,None,EZLIFT,1572,F26-641,SUGAR CANE - 3/4 SC,22.5,47,1,0,95.16
3,99 VOIDED,56,HONEYCOMB,None,294617-1,None,2117,1391,VK CUSTOM LIVING,SAMUELS-SUGAR CANE,...,None,EZLIFT,1572,F26-641,SUGAR CANE - 3/4 SC,30.5,71,5,0,799.5
4,95 PAID,56,HONEYCOMB,None,298180-1,298180,2117,1391,VK CUSTOM LIVING,RAVI SUNSET MOTOR,...,None,MOTOR,1633,F27-641,SUGAR CANE - 3/4 SC B/O,34.5,88.5,3,3,635.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,1 QUOTE,45,ROLLER SHADES,None,298193-2,None,2117,1391,VK CUSTOM LIVING,COSTA- KEKE,...,None,CLUTCH,3953,ALE111,Alexa White,45,45,1,0,133.53
87,1 QUOTE,45,ROLLER SHADES,None,298193-3,None,2117,1391,VK CUSTOM LIVING,COSTA- KEKE,...,None,CLUTCH,3953,ALE111,Alexa White,32.5,45,1,0,118.9
88,1 QUOTE,45,ROLLER SHADES,None,298193-4,None,2117,1391,VK CUSTOM LIVING,COSTA- KEKE,...,None,CLUTCH,3953,ALE111,Alexa White,32.5,45,1,0,118.9
89,1 QUOTE,45,ROLLER SHADES,None,298193-5,None,2117,1391,VK CUSTOM LIVING,COSTA- KEKE,...,None,CLUTCH,3953,ALE111,Alexa White,45,45.5,1,0,133.53
